In [37]:
import pandas as pd
import numpy as np
import os
from lxml import etree
import xml.etree.ElementTree as ET
from Bio import Entrez
from Bio import SeqIO
import time
import datetime

Entrez.email = "mnaffairs.intl@gmail.com"

In [3]:
aatranlation = {'Ala': 'A', 'Arg': 'R', 'Asn': 'N', 'Asp': 'D', 'Cys': 'C',
                'Glu': 'E', 'Gln': 'Q', 'Gly': 'G', 'His': 'H', 'Ile': 'I',
                'Leu': 'L', 'Lys': 'K', 'Met': 'M', 'Phe': 'F', 'Pro': 'P',
                'Ser': 'S', 'Thr': 'T', 'Trp': 'W', 'Tyr': 'Y', 'Val': 'V'}

In [4]:
def readHumanGenes(path):
    human_genes = []
    with open(path, 'r') as filehandle:
        human_genes = filehandle.read().splitlines()
    return human_genes

In [5]:
human_genes = readHumanGenes('../data/UniProtHumanEnzymeGenes.txt')
print(f'{len(human_genes)} number of genes are imported')

4312 number of genes are imported


In [12]:
class variationHandler(object):
    def __init__(self, enzyme_genes):
        self.dictlist = {'interpretation': [], 'gene':[], 'gene_name':[], 'accession':[], 'mutation': [], 'NP': [], 'Chr': [], 'start':[], 'stop':[], 'referenceAllele':[], 'alternateAllele':[]}
        self.enzyme_genes = enzyme_genes
        self.unnecessary_types = ('inversion', 'copy number gain', 'tandem duplication', 'microsatellite', 'copy number loss', 'distinct chromosomes', 'fusion', 'complex', 'duplication', 'translocation')
        self.is_type = False
        self.gene = ""
        self.gene_name = ""
        self.accession = ""
        self.mutation = ""
        self.np_num = ""
        self.change = ""
        self.chr = ""
        self.start_num = ""
        self.stop_num = ""
        self.referenceAllele = ""
        self.alternateAllele = ""
        self.is_GeneList = False
        self.ct_gene = 0
        self.check_grch = False
        self.ct_np = 0
        self.ct_mc = 0  # counter for Molecular Consequence tag
        self.is_haplotype = False  # check if a variation is haplotype or not
        self.is_genotype = False  # check if a variation is genotype or not
        self.is_missense = False
        self.is_conflicting = False
        self.is_not_provided = False
        self.is_interpretations = False
        self.is_interpretation = False
        self.is_description = False
        self.is_desc_hist = False
        self.intpn = []
        self.ct = 0
        self.ct_missense = 0
        self.ct_uncertain = 0
        self.ct_conflicting = 0
        self.ct_not_provided = 0
        
    def start(self, tag, attrs):
        if (tag == 'VariationArchive') and (attrs.get('VariationType').lower() not in self.unnecessary_types):
            self.is_type = True
            if attrs.get('VariationType').lower() == 'haplotype':
                self.is_haplotype = True
            if attrs.get('VariationType').lower() == 'compoundheterozygote':
                self.is_genotype = True
            self.accession = attrs.get('Accession')
        if self.is_type:
            if tag == 'GeneList':
                self.is_GeneList = True
            elif tag == 'Gene' and self.ct_gene == 0:
                self.gene = attrs.get('Symbol')
                self.gene_name = attrs.get('FullName')
                self.ct_gene += 1
            elif tag == 'SequenceLocation' and self.is_GeneList == False and self.check_grch == False:
                if attrs.get('Assembly') == 'GRCh38':
                    self.chr = attrs.get('Chr')
                    self.start_num = attrs.get('start')
                    self.stop_num = attrs.get('stop')
                    self.referenceAllele = attrs.get('referenceAlleleVCF')
                    self.alternateAllele = attrs.get('alternateAlleleVCF')
                    self.check_grch = True
            elif tag == 'ProteinExpression' and self.ct_np == 0:
                self.np_num = attrs.get('sequenceAccessionVersion')
                self.change = attrs.get('change') 
                if self.np_num and self.np_num.startswith('NP'):            
                    self.ct_np += 1
            elif tag == 'MolecularConsequence' and self.ct_mc == 0:
                if attrs.get('Type') and 'missense' in attrs.get('Type').lower():
                    self.is_missense = True
                    self.ct_missense += 1
                self.ct_mc += 1
            elif tag == 'Interpretations':
                self.is_interpretations = True
            elif tag == 'Interpretation':
                self.is_interpretation = True
            elif tag == 'Description':
                self.is_description = True
            elif tag == 'DescriptionHistory':
                self.is_desc_hist = True
            
    def end(self, tag):
        if (tag == 'VariationArchive' and self.is_type) or ((self.is_haplotype or self.is_genotype) and tag == 'SimpleAllele'):
            if len(self.intpn) == 1:
                interpretation = self.intpn[0].lower()
                if "uncertain" in interpretation:
                    self.is_uncertain = True
                    self.ct_uncertain += 1
                elif "conflicting" in interpretation:
                    self.is_conflicting = True
                    self.ct_conflicting += 1
                elif "not provided" in interpretation:
                    self.is_not_provided = True
                    self.ct_not_provided += 1
#             elif len(self.intpn) > 1:
#                 print(f'Interpretaion: {self.intpn}, Accession: {self.accession}, count: {self.ct}')
            
            if (self.gene in self.enzyme_genes) and self.is_missense and (self.is_uncertain or self.is_conflicting or self.is_not_provided):
                try:
                    self.change = self.change.split('p.')[1]
                    before = aatranlation.get(self.change[0:3])
                    after = aatranlation.get(self.change[len(self.change) - 3:len(self.change)])
                except: 
                    before = None
                    after = None
                if before and after:  # check if both have a value in aa dict
                    num = self.change[3:len(self.change) - 3]
                    abbreviated_change = before + num + after
                    fasta = np.nan
                    self.dictlist['interpretation'].append(interpretation)
                    self.dictlist['gene'].append(self.gene)
                    self.dictlist['gene_name'].append(self.gene_name)
                    self.dictlist['accession'].append(self.accession)
                    self.dictlist['mutation'].append(abbreviated_change)
                    self.dictlist['NP'].append(self.np_num)
                    self.dictlist['Chr'].append(self.chr)
                    self.dictlist['start'].append(self.start_num)
                    self.dictlist['stop'].append(self.stop_num)
                    self.dictlist['referenceAllele'].append(self.referenceAllele)
                    self.dictlist['alternateAllele'].append(self.alternateAllele)
            self.ct_gene = 0             
            self.check_grch = False
            self.is_missense = False
            self.is_uncertain = False
            self.is_conflicting = False
            self.is_not_provided = False
            self.ct_np = 0
            self.ct_mc = 0
            self.intpn = []
        if self.is_type:
            if tag == 'GeneList':
                self.is_GeneList = False
            elif tag == 'Interpretations':
                self.is_interpretations = False
            elif tag == 'Interpretation':
                self.is_interpretaion = False
            elif tag == 'Description':
                self.is_description = False
            elif tag == 'DescriptionHistory':
                self.is_desc_hist = False
        if tag == 'VariationArchive':
            self.is_type = False
            self.is_haplotype = False
            self.is_genotype = False
            self.ct +=1
            if self.ct % 10000 == 0:
                print(f'counter: {self.ct}')
                
    def data(self, data):
        if self.is_interpretations and self.is_interpretation and self.is_description and (not self.is_desc_hist):
            self.intpn.append(data)
            
    def close(self):
        print(f"Variations: {self.ct}")
        print(f"Uncertain Significance: {self.ct_uncertain}")
        print(f"Conflicting Report: {self.ct_conflicting}")
        print(f"Not Provided: {self.ct_not_provided}")
        print(f"Missense: {self.ct_missense}")
        print(f"Mutations in the list: {len(self.dictlist['gene'])}")
        print('debug: the file is closed')
        return self.dictlist

In [9]:
# read xml file of variations from ClinVar
# return dataframe and write to a csv file
def readClinVarVariationsXML(input_path, output_path, gene_set):
    print('debug: start parcing')
    parser = etree.XMLParser(target=variationHandler(gene_set))
    data = etree.parse(input_path, parser)
    df = pd.DataFrame(data)
    df.to_csv(output_path, index = False, header = True)
    return df

In [153]:
# this column is a test for a certain variation I want to look over
subnode = '../data/subnode.xml'
temp_path = '../data/temp.csv'
df_temp = readClinVarVariationsXML(subnode, temp_path, human_genes)
df_temp.head()

debug: start parcing
Interpretaion: ['Likely pathogenic', 'Pathogenic3', 'Pathogenic4'], Accession: VCV000101229, count: 0
Variations: 1
Uncertain Significance: 0
Conflicting Report: 0
Not Provided: 0
Missense: 1
debug: the file is closed


,interpretation,gene,gene_name,accession,mutation,NP,Chr,start,stop,referenceAllele,alternateAllele


In [11]:
xmlfile = '../data/ClinVarVariationRelease_00-latest_weekly.xml'
out_path = '../data/MM_enzyme.csv'
df_0 = readClinVarVariationsXML(xmlfile, out_path, human_genes)
df_0.head()

debug: start parcing
counter: 10000
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000012173, count: 12507
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000025511, count: 15858
counter: 20000
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000132775, count: 27022
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000132776, count: 27023
counter: 30000
counter: 40000
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000242384, count: 48684
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000242385, count: 48685
Interpretaion: ['no interpretation for the single variant', '

counter: 50000
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000264658, count: 53696
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000264659, count: 53697
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000264660, count: 53698
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000264668, count: 53699
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000264669, count: 53700
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000264672, count: 53701
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV00026467

Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000617633, count: 158698
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000617662, count: 158705
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000617663, count: 158706
counter: 160000
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000624596, count: 160203
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000624597, count: 160204
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000624598, count: 160205
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV0

counter: 340000
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000446183, count: 341316
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000446188, count: 341317
counter: 350000
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000692326, count: 351889
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000692327, count: 351890
counter: 360000
counter: 370000
counter: 380000
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000242396, count: 385171
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000242500, count: 385172
Interpretaion: ['no interpretation for the single varian

Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000267744, count: 412656
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000267745, count: 412657
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000375728, count: 418223
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000375730, count: 418224
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000417816, count: 419793
counter: 420000
Interpretaion: ['Pathogenic', 'Pathogenic'], Accession: VCV000424757, count: 420476
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000431012, count: 421215
Interpretaion: ['Pathogenic', 'Confl

...

Interpretaion: ['no interpretation for the single variant', 'drug response', 'no interpretation for the single variant', 'drug response', 'drug response'], Accession: VCV000634287, count: 653777
Interpretaion: ['no interpretation for the single variant', 'drug response', 'no interpretation for the single variant', 'drug response', 'drug response'], Accession: VCV000634288, count: 653778
Interpretaion: ['no interpretation for the single variant', 'drug response', 'no interpretation for the single variant', 'drug response', 'drug response'], Accession: VCV000634289, count: 653779
Interpretaion: ['no interpretation for the single variant', 'drug response', 'no interpretation for the single variant', 'drug response', 'drug response'], Accession: VCV000634290, count: 653780
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant', 'drug response', 'no interpretation for the single variant', 'drug response', 'drug response'], Accession: VCV000634

Interpretaion: ['no interpretation for the single variant', 'drug response', 'no interpretation for the single variant', 'drug response', 'drug response'], Accession: VCV000634396, count: 653886
Interpretaion: ['no interpretation for the single variant', 'drug response', 'no interpretation for the single variant', 'drug response', 'drug response'], Accession: VCV000634397, count: 653887
Interpretaion: ['no interpretation for the single variant', 'drug response', 'no interpretation for the single variant', 'drug response', 'drug response'], Accession: VCV000634398, count: 653888
Interpretaion: ['no interpretation for the single variant', 'drug response', 'no interpretation for the single variant', 'drug response', 'drug response'], Accession: VCV000634399, count: 653889
Interpretaion: ['no interpretation for the single variant', 'drug response', 'no interpretation for the single variant', 'drug response', 'drug response'], Accession: VCV000634400, count: 653890
Interpretaion: ['no inter

Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000830334, count: 654764
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000830335, count: 654765
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000830336, count: 654766
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000830337, count: 654767
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000830338, count: 654768
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000830339, count: 654769
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000830340, count:

counter: 680000
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000424703, count: 680737
Interpretaion: ['Pathogenic', 'Pathogenic'], Accession: VCV000424711, count: 680739
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000425600, count: 680777
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000425651, count: 680779
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000440737, count: 681766
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000440739, count: 681767
Interpretaion: ['no interpretation for the single variant', 'no interpretation for the single variant'], Accession: VCV000446185, count: 681770
Interpretaion: ['no interpretation f

,interpretation,gene,gene_name,accession,mutation,NP,Chr,start,stop,referenceAllele,alternateAllele
0,uncertain significance,AGA,aspartylglucosaminidase,VCV000000222,G60D,NP_000018.2,4,177440375,177440375,C,T
1,uncertain significance,DPYD,dihydropyrimidine dehydrogenase,VCV000000437,R886H,NP_000101.2,1,97098598,97098598,C,T
2,uncertain significance,PTS,6-pyruvoyltetrahydropterin synthase,VCV000000477,R16C,NP_000308.1,11,112226489,112226489,C,T
3,uncertain significance,PROC,"protein C, inactivator of coagulation factors ...",VCV000000661,P210L,NP_000303.1,2,127426178,127426178,C,T
4,uncertain significance,ARSB,arylsulfatase B,VCV000000877,G137V,NP_000037.2,5,78969095,78969095,C,A


In [7]:
class variationHandlerSpecific(object):
    def __init__(self, accession):
        self.is_accession = False
        self.accession = accession
        self.ct = 0
        print(self.accession)
        
    def start(self, tag, attrs):
        global WFILE
        if (tag == 'VariationArchive') and (attrs.get('Accession') == self.accession):
            self.is_accession = True
            print('The specific variation is found: ' + str(self.ct))
        if self.is_accession:
            if len(attrs.keys()) == 0:
                WFILE.write('<' + tag)
            else:   
                for i, t in enumerate(attrs.keys()):
                    if i == 0:
                        WFILE.write('<' + tag + ' ')
                    elif i != len(attrs.keys()) - 1:
                        WFILE.write(t + '="' + attrs.get(t) + '"' + ' ')
                    else:
                        WFILE.write(t + '="' + attrs.get(t) + '"')
            WFILE.write('>')
            
    def end(self, tag):
        global WFILE
        if self.is_accession:
            WFILE.write('</' + tag + '>')
        if tag == 'VariationArchive':
            if self.ct % 10000 == 0:
                print(self.ct)
            self.ct += 1
        if self.is_accession and tag == 'VariationArchive':
            self.is_accession = False
            WFILE.close()
            print('The subnode file is completed')
            
    def data(self, data):
        global WFILE
        if data is not None:
            if self.is_accession and data != "":
                WFILE.write(data)
            
    def close(self):
        print('The xml file is closed')

In [8]:
# read xml file of variations from ClinVar
# return dataframe and write to a csv file
def readClinVarVariationsXMLSpecific(input_path, accession):
    print('Start parcing')
    parser = etree.XMLParser(target=variationHandlerSpecific(accession))
    etree.parse(input_path, parser)

In [11]:
# this column is to get a certain part of the xml file
WFILE = open('../data/subnode.xml', 'w')
xmlfile = '../data/ClinVarVariationRelease_00-latest_weekly.xml'
accession = 'VCV000065613'
readClinVarVariationsXMLSpecific(xmlfile, accession)

Start parcing
VCV000065613
0
10000
20000
The specific variation is found: 20707
The subnode file is completed
30000
40000
The xml file is closed


KeyboardInterrupt: 

In [6]:
# makes dictionary of fasta sequences and np number 
# returns the dictionary
def makeDictOfFasta(dictpath):
    fasta_dict = {}
    for root, d_names, file_names in os.walk(dictpath):
        for filename in file_names:
            fname = os.path.join(root, filename)
            with open(fname, 'r') as f:
                print('opened a fasta file')
                np_num = ''
                sequence = ''
                for line in f:
                    if line[0] == '>':
                        if sequence != '':
                            fasta_dict[np_num] = sequence
                            np_num = ''
                            sequence = ''                    
                        i = 1
                        while line[i] != ' ':
                            np_num += line[i]
                            i += 1
                    else:
                        line = line.strip('\n')
                        sequence += line
    print(f'length of fasta dictionary: {len(fasta_dict)}')
    return fasta_dict

In [7]:
fasta_dict = makeDictOfFasta('../fasta_sequences/')

opened a fasta file
opened a fasta file
opened a fasta file
opened a fasta file
opened a fasta file
opened a fasta file
opened a fasta file
length of fasta dictionary: 113645


In [8]:
print(fasta_dict.get('NP_037514.2'))

MPPATGGGLAESELRPRRGRCGPQAARAAGRDVAAEAVARSPKRPAWGSRRFEAVGWWALLALVTLLSFATRFHRLDEPPHICWDETHFGKMGSYYINRTFFFDVHPPLGKMLIGLAGYLSGYDGTFLFQKPGDKYEHHSYMGMRGFCAFLGSWLVPFAYLTVLDLSKSLSAALLTAALLTFDTGCLTLSQYILLDPILMFFIMAAMLSMVKYNSCADRPFSAPWWFWLSLTGVSLAGALGVKFVGLFIILQVGLNTIADLWYLFGDLSLSLVTVGKHLTARVLCLIVLPLALYTATFAVHFMVLSKSGPGDGFFSSAFQARLSGNNLHNASIPEHLAYGSVITVKNLRMAIGYLHSHRHLYPEGIGARQQQVTTYLHKDYNNLWIIKKHNTNSDPLDPSFPVEFVRHGDIIRLEHKETSRNLHSHYHEAPMTRKHYQVTGYGINGTGDSNDFWRIEVVNRKFGNRIKVLRSRIRFIHLVTGCVLGSSGKVLPKWGWEQLEVTCTPYLKETLNSIWNVEDHINPKLPNISLDVLQPSFPEILLESHMVMIRGNSGLKPKDNEFTSKPWHWPINYQGLRFSGVNDTDFRVYLLGNPVVWWLNLLSIALYLLSGSIIAVAMQRGARLPAEVAGLSQVLLRGGGQVLLGWTLHYFPFFLMGRVLYFHHYFPAMLFSSMLTGILWDTLLRLCAWGLASWPLARGIHVAGILSLLLGTAYSFYLFHPLAYGMVGPLAQDPQSPMAGLRWLDSWDF


In [13]:
# crops fasta sequence
# returns the cropped sequnece with a specified range
def cropFASTA(sequence, location, reference, seqRange):
    if location - 1 < len(sequence) and sequence[location - 1] == reference:
        if location - 1 - seqRange <= 0:
            proteinSeq = sequence[0:2 * seqRange + 1]
        elif location - 1 + seqRange > len(sequence) - 1:
            proteinSeq = sequence[0 if len(sequence) - 1 - 2 * seqRange <= 0 else len(sequence) - 1 - 2 * seqRange:len(sequence)]
        else:
            proteinSeq = sequence[location - 1 - seqRange : location + seqRange]
        return proteinSeq
    else:
        return None 

In [26]:
def addFASTAfromDict(fasta_dict, df):
    unfound = set()
    seq_list = []
    for index, row in df.iterrows():
        mutation = row['mutation']
        try:
            ref = mutation[0]
            try:
                location = int(mutation[1:len(mutation)-1])
            except:
                location = int(mutation.split('_')[0][1:])
            np_num = row['NP']  # specify the column of np 
            sequence = fasta_dict[np_num]  # 
            seqRange = 12  # range of sequences to take
            seq = cropFASTA(sequence, location, ref, seqRange) if sequence else None
        except:
            seq = None
            unfound_np = row['NP']
            unfound.add(unfound_np)
        seq_list.append(seq)
    df['FASTA'] = seq_list
    print(f'Unfound Sequences: {len(unfound)} {unfound}')
    return df

In [24]:
df_0 = pd.read_csv('../data/MM_enzyme.csv')
df_0.head()

,interpretation,gene,gene_name,accession,mutation,NP,Chr,start,stop,referenceAllele,alternateAllele,FASTA
0,uncertain significance,AGA,aspartylglucosaminidase,VCV000000222,G60D,NP_000018.2,4,177440375,177440375,C,T,ASGGSALDAVESGCAMCEREQCDGS
1,uncertain significance,DPYD,dihydropyrimidine dehydrogenase,VCV000000437,R886H,NP_000101.2,1,97098598,97098598,C,T,KKLPSFGPYLEQRKKIIAENKIRLK
2,uncertain significance,PTS,6-pyruvoyltetrahydropterin synthase,VCV000000477,R16C,NP_000308.1,11,112226489,112226489,C,T,EGGGRRCQAQVSRRISFSASHRLYS
3,uncertain significance,PROC,"protein C, inactivator of coagulation factors ...",VCV000000661,P210L,NP_000303.1,2,127426178,127426178,C,T,KRDTEDQEDQVDPRLIDGKMTRRGD
4,uncertain significance,ARSB,arylsulfatase B,VCV000000877,G137V,NP_000037.2,5,78969095,78969095,C,A,DEKLLPQLLKEAGYTTHMVGKWHLG


In [27]:
df_1 = addFASTAfromDict(fasta_dict, df_0)
df_1.to_csv('../data/MM_enzyme.csv', index = False, header = True)
df_1.head()

Unfound Sequences: 23 {'NP_001180240.1', 'LRG_789p1', 'NP_000450.2', 'NP_001276896.1', 'NP_004597.2', 'NP_001202.4', 'NP_000114.2', 'NP_001017973.1', 'NP_000342.2', 'NP_065829.3', 'NP_001138503.1', 'NP_004251.3', 'NP_001273003.1', 'NP_945347.2', 'NP_000274.2', 'NP_000299.2', 'NP_001191354.1', 'NP_001017975.4', 'NP_001177317.1', 'NP_001041636.1', 'LRG_319p1', 'NP_000439.1', 'NP_000195.2'}


,interpretation,gene,gene_name,accession,mutation,NP,Chr,start,stop,referenceAllele,alternateAllele,FASTA
0,uncertain significance,AGA,aspartylglucosaminidase,VCV000000222,G60D,NP_000018.2,4,177440375,177440375,C,T,ASGGSALDAVESGCAMCEREQCDGS
1,uncertain significance,DPYD,dihydropyrimidine dehydrogenase,VCV000000437,R886H,NP_000101.2,1,97098598,97098598,C,T,KKLPSFGPYLEQRKKIIAENKIRLK
2,uncertain significance,PTS,6-pyruvoyltetrahydropterin synthase,VCV000000477,R16C,NP_000308.1,11,112226489,112226489,C,T,EGGGRRCQAQVSRRISFSASHRLYS
3,uncertain significance,PROC,"protein C, inactivator of coagulation factors ...",VCV000000661,P210L,NP_000303.1,2,127426178,127426178,C,T,KRDTEDQEDQVDPRLIDGKMTRRGD
4,uncertain significance,ARSB,arylsulfatase B,VCV000000877,G137V,NP_000037.2,5,78969095,78969095,C,A,DEKLLPQLLKEAGYTTHMVGKWHLG


In [44]:
def getFASTA(ls_np):
    fasta_dict = {}
    for np_num in ls_np:
        handle = Entrez.efetch(db='protein', id=np_num, rettype='fasta', retmode='text', api_key='2959e9bc88ce27224b70cada27e5b58c6b09')
        seq_record = SeqIO.read(handle, 'fasta')
        sequence = seq_record.seq
        fasta_dict[np_num] = sequence
    return fasta_dict

In [45]:
unfound_nps = ('NP_001180240.1', 'NP_000450.2', 'NP_001276896.1', 'NP_004597.2', 'NP_001202.4', 'NP_000114.2', 'NP_001017973.1', 'NP_000342.2', 'NP_065829.3', 'NP_001138503.1', 'NP_004251.3', 'NP_001273003.1', 'NP_945347.2', 'NP_000274.2', 'NP_000299.2', 'NP_001191354.1', 'NP_001017975.4', 'NP_001177317.1', 'NP_001041636.1', 'NP_000439.1', 'NP_000195.2')
add_fasta = getFASTA(unfound_nps)
fasta_dict.update(add_fasta)
      

113666


In [46]:
df_1 = addFASTAfromDict(fasta_dict, df_0)
df_1.to_csv('../data/MM_enzyme.csv', index = False, header = True)
df_1.head()

Unfound Sequences: 2 {'LRG_319p1', 'LRG_789p1'}


,interpretation,gene,gene_name,accession,mutation,NP,Chr,start,stop,referenceAllele,alternateAllele,FASTA
0,uncertain significance,AGA,aspartylglucosaminidase,VCV000000222,G60D,NP_000018.2,4,177440375,177440375,C,T,ASGGSALDAVESGCAMCEREQCDGS
1,uncertain significance,DPYD,dihydropyrimidine dehydrogenase,VCV000000437,R886H,NP_000101.2,1,97098598,97098598,C,T,KKLPSFGPYLEQRKKIIAENKIRLK
2,uncertain significance,PTS,6-pyruvoyltetrahydropterin synthase,VCV000000477,R16C,NP_000308.1,11,112226489,112226489,C,T,EGGGRRCQAQVSRRISFSASHRLYS
3,uncertain significance,PROC,"protein C, inactivator of coagulation factors ...",VCV000000661,P210L,NP_000303.1,2,127426178,127426178,C,T,KRDTEDQEDQVDPRLIDGKMTRRGD
4,uncertain significance,ARSB,arylsulfatase B,VCV000000877,G137V,NP_000037.2,5,78969095,78969095,C,A,DEKLLPQLLKEAGYTTHMVGKWHLG


In [39]:
df_1[['NP', 'gene', 'gene_name', 'FASTA']].head()

,NP,gene,gene_name,FASTA
0,NP_000018.2,AGA,aspartylglucosaminidase,ASGGSALDAVESGCAMCEREQCDGS
1,NP_000101.2,DPYD,dihydropyrimidine dehydrogenase,KKLPSFGPYLEQRKKIIAENKIRLK
2,NP_000308.1,PTS,6-pyruvoyltetrahydropterin synthase,EGGGRRCQAQVSRRISFSASHRLYS
3,NP_000303.1,PROC,"protein C, inactivator of coagulation factors ...",KRDTEDQEDQVDPRLIDGKMTRRGD
4,NP_000037.2,ARSB,arylsulfatase B,DEKLLPQLLKEAGYTTHMVGKWHLG


In [47]:
# make FASTA format text file from dataframe for blast search
def makeFASTAfile(df, output_path):
    subset = df[['NP', 'gene', 'gene_name', 'FASTA']]
    tuples = [tuple(x) for x in subset.values]
    with open(output_path, 'w') as f:
        for tup in tuples:
            line = '>' + '\t'.join(tup[0:3]) + '\n'
            fasta = str(tup[3]) + '\n'
            f.write(line)
            f.write(fasta) 

In [48]:
makeFASTAfile(df_1, '../data/fasta.txt')

In [49]:
def blastLocal(fasta_path, out_path, evalue=10.0, window_size=3):
    cmd1 = '../ncbi/blast/bin/'
    cmd2 = 'blastp' + ' '\
         + '-query ' + fasta_path + ' '\
         + '-db ' + cmd1 + 'pdbaa' + ' '\
         + '-evalue ' + str(evalue) + ' '\
         + '-outfmt ' + '5' + ' '\
         + '-out ' + out_path
    cmd = cmd1 + cmd2
    b_cmd = os.system(cmd)
    print(cmd + ' : ran with exit code %d' %b_cmd)

In [50]:
start = datetime.datetime.now()

fasta_path = '../data/fasta.txt'  # path for the fasta file
out_path = '../data/blast_result.xml'  # path for the output file
evalue = 10.0
size = 3
blastLocal(fasta_path, out_path, evalue, size)

end = datetime.datetime.now()
time = end - start
c = divmod(time.days * 86400 + time.seconds, 60)
print(c)

../ncbi/blast/bin/blastp -query ../data/fasta.txt -db ../ncbi/blast/bin/pdbaa -evalue 10.0 -outfmt 5 -out ../data/blast_result.xml : ran with exit code 0
(24, 33)


In [51]:
class BlastHandler(object):
    def __init__(self):
        self.dictlist = {'pdb': [], 'evalue': [], 'hit_from': [], 'hit_to': []}
        self.is_hit_id = False
        self.is_evalue = False
        self.is_hit_from = False
        self.is_hit_to = False
        self.ct_iter = 0
        self.ct = 0
        
    def start(self, tag, attrs):
        if tag == 'Hit':  # there are a few <Hsp> tags within a <Hit> tag
            self.ct_iter += 1
        elif tag == 'Hit_id':
            self.is_hit_id = True
        elif tag == 'Hsp_evalue':
            self.is_evalue = True
        elif tag == 'Hsp_hit-from':
            self.is_hit_from = True
        elif tag == 'Hsp_hit-to':
            self.is_hit_to = True
            
    def end(self, tag):
        if tag == 'Iteration':
            if self.ct_iter == 0:  # when there is no hit, add None to each list
                self.dictlist['pdb'].append(None)
                self.dictlist['evalue'].append(None)
                self.dictlist['hit_from'].append(None)
                self.dictlist['hit_to'].append(None)
            self.ct_iter = 0
            self.ct += 1
            if self.ct % 10000 == 0:
                print(f'coutner: {self.ct}')
                print(f'Length pdb: {len(self.dictlist["pdb"])}, evalue: {len(self.dictlist["evalue"])}, hit_from : {len(self.dictlist["hit_from"])}, hit_to : {len(self.dictlist["hit_to"])}')
        elif tag == 'Hit_id':
            self.is_hit_id = False
        elif tag == 'Hsp_evalue':
            self.is_evalue = False
        elif tag == 'Hsp_hit-from':
            self.is_hit_from = False
        elif tag == 'Hsp_hit-to':
            self.is_hit_to = False
        elif tag == 'Hsp':
            self.ct_iter += 1
        
    def data(self, data):
        if self.ct_iter == 1:
            if self.is_hit_id:
                try:
                    pdb = data.split('pdb|')[1]
                except: 
                    print(f'Cannot split pdb {data}')
                    pdb = None
                self.dictlist['pdb'].append(pdb)
            elif self.is_evalue:
                self.dictlist['evalue'].append(data)
            elif self.is_hit_from:
                self.dictlist['hit_from'].append(data)
            elif self.is_hit_to:
                self.dictlist['hit_to'].append(data)
    
    def close(self):
        print('Has completed parsing the blast_result xml')
        print(f'Total Count: {self.ct}')
        print(f'Length pdb: {len(self.dictlist["pdb"])}, evalue: {len(self.dictlist["evalue"])}, hit_from : {len(self.dictlist["hit_from"])}, hit_to : {len(self.dictlist["hit_to"])}')
        return self.dictlist

In [52]:
def readBlastXML(input_path):
    print('Start parcing')
    parser = etree.XMLParser(target=BlastHandler())
    data = etree.parse(input_path, parser)
    return data    

In [53]:
def combineBlastResults(df, dictlist):
    df['pdb'] = dictlist['pdb']
    df['evalue'] = dictlist['evalue']
    df['hit_from'] = dictlist['hit_from']
    df['hit_to'] = dictlist['hit_to']
    return df

In [54]:
blast_dl = readBlastXML('../data/blast_result.xml')
blast_df = pd.DataFrame(blast_dl)
print(len(blast_dl['pdb']))
blast_df.head()

Start parcing
coutner: 10000
Length pdb: 10000, evalue: 10000, hit_from : 10000, hit_to : 10000
coutner: 20000
Length pdb: 20000, evalue: 20000, hit_from : 20000, hit_to : 20000
coutner: 30000
Length pdb: 30000, evalue: 30000, hit_from : 30000, hit_to : 30000
coutner: 40000
Length pdb: 40000, evalue: 40000, hit_from : 40000, hit_to : 40000
coutner: 50000
Length pdb: 50000, evalue: 50000, hit_from : 50000, hit_to : 50000
coutner: 60000
Length pdb: 60000, evalue: 60000, hit_from : 60000, hit_to : 60000
Has completed parsing the blast_result xml
Total Count: 69791
Length pdb: 69791, evalue: 69791, hit_from : 69791, hit_to : 69791
69791


,pdb,evalue,hit_from,hit_to
0,1APY|A,6.44088e-11,25,49
1,1GT8|A,1.87182e-09,874,898
2,3I2B|A,1.89206e-10,3,24
3,4DT7|E,3.1355e-07,1,19
4,1FSU|A,3.56958e-11,84,108


In [55]:
df_1 = pd.read_csv('../data/MM_enzyme.csv')
df_2 = combineBlastResults(df_1, blast_dl)
df_2.to_csv('../data/MM_enzyme_blast.csv', index = False, header = True)
df_2.head()

,interpretation,gene,gene_name,accession,mutation,NP,Chr,start,stop,referenceAllele,alternateAllele,FASTA,pdb,evalue,hit_from,hit_to
0,uncertain significance,AGA,aspartylglucosaminidase,VCV000000222,G60D,NP_000018.2,4,177440375,177440375,C,T,ASGGSALDAVESGCAMCEREQCDGS,1APY|A,6.44088e-11,25,49
1,uncertain significance,DPYD,dihydropyrimidine dehydrogenase,VCV000000437,R886H,NP_000101.2,1,97098598,97098598,C,T,KKLPSFGPYLEQRKKIIAENKIRLK,1GT8|A,1.87182e-09,874,898
2,uncertain significance,PTS,6-pyruvoyltetrahydropterin synthase,VCV000000477,R16C,NP_000308.1,11,112226489,112226489,C,T,EGGGRRCQAQVSRRISFSASHRLYS,3I2B|A,1.89206e-10,3,24
3,uncertain significance,PROC,"protein C, inactivator of coagulation factors ...",VCV000000661,P210L,NP_000303.1,2,127426178,127426178,C,T,KRDTEDQEDQVDPRLIDGKMTRRGD,4DT7|E,3.1355e-07,1,19
4,uncertain significance,ARSB,arylsulfatase B,VCV000000877,G137V,NP_000037.2,5,78969095,78969095,C,A,DEKLLPQLLKEAGYTTHMVGKWHLG,1FSU|A,3.56958e-11,84,108


In [23]:
df_2 = pd.read_csv('../data/MM_enzyme_blast.csv')
df_2[df_2.duplicated(['accession'])]

,interpretation,gene,gene_name,accession,mutation,NP,Chr,start,stop,referenceAllele,alternateAllele,FASTA,pdb,evalue,hit_from,hit_to
31612,conflicting interpretations of pathogenicity,MOCS1,molybdenum cofactor synthesis 1,VCV000208497,R339W,NP_001345458.1,6,39909922,39909922,G,A,CLFGNSEVSLRDHLRAGASEQ,4JAX|A,5.439420e+00,114.0,125.0
34310,conflicting interpretations of pathogenicity,GBA,glucosylceramidase beta,VCV000004299,D179H,NP_000148.2,1,155238570,155238570,C,G,IRTYTYADTPDDFQLHNFSLP,2V3D|A,7.380430e-09,132.0,152.0
38043,conflicting interpretations of pathogenicity,ABCA4,ATP binding cassette subfamily A member 4,VCV000812193,R1898H,NP_000341.2,1,94010821,94010821,C,T,VYFLLTLLVQRHFFLSQWIAE,5XJY|A,3.338630e+00,1883.0,1897.0
39804,not provided,CFTR,CF transmembrane conductance regulator,VCV000007239,G1244V,NP_000483.3,7,117642451,117642451,G,T,ISPGQRVGLLGRTGSGKSTLL,3GD7|A,5.697710e-08,44.0,64.0
48289,uncertain significance,BRCA1,BRCA1 DNA repair associated,VCV000624568,G1809D,NP_009231.2,17,43049164,43049164,C,T,DQLEWMVQLCGASVVKELSSF,3COJ|A,1.006400e-08,154.0,174.0
51431,uncertain significance,BTD,biotinidase,VCV000038564,R59H,NP_001268652.2,3,15635615,15635615,G,A,LSLNPLALISRQEALELMNQN,4CYG|A,1.492640e+00,34.0,49.0
52006,uncertain significance,TERT,telomerase reverse transcriptase,VCV000036946,V867M,NP_001180305.1,5,1266519,1266519,C,T,IRRDGLLLRLVDDFLLVTPHL,1TP6|A,3.799470e-01,30.0,46.0
54243,uncertain significance,EXT2,exostosin glycosyltransferase 2,VCV000218894,R128C,NP_000392.3,11,44107995,44107995,C,T,CRMHTCFDVYRCGFNPKNKIK,1W9P|A,5.956950e+00,276.0,292.0
56946,conflicting interpretations of pathogenicity,POLG,"DNA polymerase gamma, catalytic subunit",VCV000424791,R852C,NP_001119603.1,15,89321780,89321780,G,A,PQVVTAGTITRRAVEPTWLTA,4ZTU|A,1.838480e-07,814.0,834.0
56949,uncertain significance,TTN,titin,VCV000424837,K33744T,NP_001243779.1,2,178530461,178530461,T,G,MGQAFKSIHEKVSKISETKKS,NaN,NaN,NaN,NaN
